In [9]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
import netCDF4 as nc4

import xarray as xr

In [10]:
pmm = "netcdf_files/eORCA1.4.2_ModifStraits_domain_cfg.nc"

mm = nc4.Dataset(pmm)

glamt = mm.variables['glamt'][0] ; e1t = mm.variables['e1t'][0]
gphit = mm.variables['gphit'][0] ; e2t = mm.variables['e2t'][0] 
# U-V facets
e3t_1d = mm.variables['e3t_1d'][0] ; e3t_0 = mm.variables['e3t_0'][0] ; e3u_0 = mm.variables['e3u_0'][0]; e3v_0 = mm.variables['e3v_0'][0]
# partial cells
e3w_0 = mm.variables['e3w_0'][0] ; e3uw_0 = mm.variables['e3uw_0'][0] ; e3vw_0 = mm.variables['e3vw_0'][0]
e3w_1d = mm.variables['e3w_1d'][0]
#
totl = mm.variables['top_level'][0]  # 
botl = mm.variables['bottom_level'][0]  # goes from 0 to 74 (75 levels)
bath = mm.variables['bathy_meter'][0]   # 
nK,nJ,nI = np.shape(e3w_0)
mm.close()

e3tp = np.copy(e3t_0)

---
### e3u=moy(e3t)

In [15]:
""" 4.1) e3u,v moyenne
"""
e3up = np.copy(e3u_0) ; 
for j in range(1,nJ):
    for i in range(0,nI-1):
        if totl[j,i+1]+totl[j,i] !=2 : continue
        e3up[:,j,i] = 0.5 * ( e3tp[:,j  ,i+1] + e3tp[:,j,i] ) # moyenne i
        
i = nI-1 # east indian ocean
for j in range(nJ):
    if totl[j,0]+totl[j,i] !=2 : continue
    e3up[:,j,i] = 0.5 * ( e3tp[:,j,0] + e3tp[:,j,i] )
    
e3vp = np.copy(e3v_0)
for j in range(0,nJ-1):
    for i in range(0,nI):
        if totl[j+1,i]+totl[j,i] !=2 : continue
        e3vp[:,j,i] = 0.5 * ( e3tp[:,j+1,i  ] + e3tp[:,j,i] ) # moyenne j

j = nJ-1 # Arctique # jj = 0 Antartique
#slice at cell 180(NEMO)->179(netcdf,python) on a U face (Canada)
for i in range(nI):
    c4 = (j,180-(i-179))  #   | 4  |   
    c1 = (j,i          )  #   --+--|   
    #                     #   | 1  | 
    if totl[c4[0],c4[1]]+totl[c1[0],c1[1]] !=2 : continue
    e3vp[:,j,i] = 0.5 * ( e3tp[:,c4[0],c4[1]] \
                        + e3tp[:,c1[0],c1[1]] )
flagU="e3u moyenne"
psave = "penalised_domain/eORCA1.4.2_ModifStraits_domain_cfg_ref_pcmoy.nc"

---
### e3u = min(e3t)

In [16]:
""" 4.2) e3u,v minimum
"""
e3up = np.copy(e3u_0) ; 
for j in range(1,nJ):
    for i in range(0,nI-1):
        if totl[j,i+1]+totl[j,i] !=2 : continue
        e3up[:,j,i] = np.min([e3tp[:,j,i+1],e3tp[:,j,i]],axis=0) # min i
        
i = nI-1 # east indian ocean
for j in range(nJ):
    if totl[j,0]+totl[j,i] !=2 : continue
    e3up[:,j,i] = np.min([e3tp[:,j,0],e3tp[:,j,i]],axis=0) 
    
e3vp = np.copy(e3v_0)
for j in range(0,nJ-1):
    for i in range(0,nI):
        if totl[j+1,i]+totl[j,i] !=2 : continue
        e3vp[:,j,i] = np.min([e3tp[:,j+1,i],e3tp[:,j,i]],axis=0) # min j

j = nJ-1 # Arctique # jj = 0 Antartique
#slice at cell 180(NEMO)->179(netcdf,python) on a U face (Canada)
for i in range(nI):
    c4 = (j,180-(i-179))  #   | 4  |   
    c1 = (j,i          )  #   --+--|   
    #                     #   | 1  | 
    if totl[c4[0],c4[1]]+totl[c1[0],c1[1]] !=2 : continue
    e3vp[:,j,i] = np.min([e3tp[:,c4[0],c4[1]],
                          e3tp[:,c1[0],c1[1]]],axis=0)
flagU="e3u minimum"
psave = "penalised_domain/eORCA1.4.2_ModifStraits_domain_cfg_ref_pcmin.nc"

In [21]:
#c,b,a = np.where(e3u_0!=e3up)
# seul incoherence

---
### Write

In [14]:
""" 5) write
"""
#psave = "penalised_domain/test.nc"
#psave = "penalised_domain/eORCA1.4.2_ModifStraits_domain_cfg_ref_smoothed.nc"

ds=xr.open_dataset(pmm)

ds.e3u_0[0]=e3up
ds.e3v_0[0]=e3vp

ds.to_netcdf(psave) # rewrite to netcd
ds.close()
print("written : %s" % (psave))
print(flagU)

written : penalised_domain/eORCA1.4.2_ModifStraits_domain_cfg_ref_pcmin.nc
e3u minimum
